In [ ]:
%load_ext autoreload
%autoreload 2

import deltas
from deltas.pipeline import data, classifier, evaluation
from deltas.model import downsample
import deltas.classifiers.models as models

In [ ]:
dataset = 'MIMIC-IV'  # change ind to select dataset to use
model = 'SVM-rbf'
model = 'MLP-small'
# model = 'Linear'

# dataset = datasets[7]  # change ind to select dataset to use
data_clf = data.get_real_dataset(dataset, scale=True)

In [ ]:
data = data_clf['data']
clf = models.NN(hidden_layer_sizes=(
    10, 10), max_iter=10).fit(data['X'], data['y'])

In [ ]:
X = data_clf['data']['X']
y = data_clf['data']['y']
# deltas_model = model_deltas.reprojection_deltas(
deltas_model = downsample.downsample_deltas(clf).fit(X, y,
                                                     alpha=10,
                                                     _print=True,
                                                     _plot=True,
                                                     method='supports-prop-update_mean',
                                                     max_trials=1000,
                                                     parallel=True)

In [ ]:
classifiers_dict = {'Original': clf, 'Deltas': deltas_model}
evaluation.eval_test(classifiers_dict, data_clf['data_test'], _print=True,
                     _plot=True, dim_reducer=data_clf['dim_reducer'])